In [1]:
# 爬取數據新聞新聞記者名單和信息
import csv
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time



def get_dynamic_page_script(url):
    browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(2)
    h = browser.find_element_by_css_selector('html')
    t = h.get_attribute('innerHTML')
    browser.quit()
    return t
#這個網站用requests拿不到腳本，因此用selenium


def scrape_directory(page_script):
    mypage=BeautifulSoup(page_script,"html.parser")
    journalists_data = []
    for i in mypage.find_all('div', attrs={'class':"back"}):
        journalist = []
        journalist.append(i.find('div', attrs={'class':"name"}).text)
        journalist.append(i.find('div', attrs={'class':"institution"}).text)
        journalist.append(i.find('div', attrs={'class':"city"}).text)
        journalist.append(i.find('div', attrs={'class':"github"}).a['href'])
# 還有這些信息，暫時和後面爬github的contribution無關，先忽略
#         journalist.append(i.find('div', attrs={'class':"pgp"}).a.text)
#         journalist.append(i.find('div', attrs={'class':"email"}).a['href'].replace('mailto:', ''))
#         journalist.append(i.find('div', attrs={'class':"twitter"}).a['href'])
#         journalist.append(i.find('div', attrs={'class':"website"}).a['href'])
# 動態爬尋的速度太慢了，最後還是選擇用靜態爬
#     for i in browser.find_elements_by_class_name("back"):
#         journalist = []
#         journalist.append(i.find_element_by_class_name("name").text)
#         journalist.append(i.find_element_by_class_name("institution").text)
#         journalist.append(i.find_element_by_class_name("city").text)
#         journalist.append(i.find_element_by_xpath("//div[@class='pgp']/a").text)
#         journalist.append(i.find_element_by_xpath("//div[@class='email']/a").get_attribute('href').replace('mailto:', ''))
#         journalist.append(i.find_element_by_xpath("//div[@class='twitter']/a").get_attribute('href'))
#         journalist.append(i.find_element_by_xpath("//div[@class='github']/a").get_attribute('href'))
#         journalist.append(i.find_element_by_xpath("//div[@class='website']/a").get_attribute('href'))
#     journalists_data.append(journalist)
    
    #因為原網站已經有N/A了，所以在這裡把空信息轉化成N/A，統一格式
        for i, value in enumerate(journalist):
            if value == '':
                journalist[i]='N/A'
        journalists_data.append(journalist)
    return journalists_data



url = 'http://jplusplus.github.io/global-directory/'
page_script = get_dynamic_page_script(url)
journalists_data = scrape_directory(page_script)
with open('journalists-data.csv','w') as f:
    csv.writer(f).writerows(journalists_data)
# 輸出一個記者基本信息表csv

In [1]:
%%time
# 根據之前導出的csv中每位記者的github網址，爬取他們的contribution歷史
import csv
import requests
from bs4 import BeautifulSoup
import datetime 
import time


def scrape_github_contributions(homepage_url):
#今後只要有一個人的github的homepage_url,就可以爬取所有contribution了
    def scrpe_year_urls(homepage_url):
        year_urls = [homepage_url] 
        #2018年（今年）在homepage上可以截止到今天，但如果載入html中的地址，會截止到今年12-31（未來），所以先把首頁加進去
        r = requests.get(homepage_url).text
        home_page_script = BeautifulSoup(r,"html.parser")
        for i in home_page_script.find_all('a',attrs={'class':"js-year-link filter-item px-3 mb-2 py-2 "}):
        #這個class不包含2018年（今年）
            url ='{}{}'.format('https://github.com',i['href'])
            year_urls.append(url)
        return year_urls
    
    def scrape_one_year_contributions(url):
        r = requests.get(url).text
        page_script = BeautifulSoup(r,"html.parser")
        all_rects = page_script.find_all('rect')
        for i in reversed(all_rects):
            contributions[i['data-date']] = i['data-count']
    # 因為不同年份的每一頁在12.30這一天顯示有重複（每一年都會顯示到去年的12.30左右），所以用dict的key的不可重複性避免這個問題   
            
    contributions = {} 
    for i in scrpe_year_urls(homepage_url): 
        scrape_one_year_contributions(i)
    contributions_list = list(contributions.values())
    #dict.values輸出的不是list，要重新列表化，方便之後輸出
    return contributions_list



def write_head(beginDate): #添加標題行
    def dateRange(beginDate, endDate): #引用網絡上的，黑箱！
        dates = []
        dt = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
        date = beginDate[:]
        while date <= endDate:
            dates.append(date)
            dt = dt + datetime.timedelta(1)
            date = dt.strftime("%Y-%m-%d")
        return reversed(dates)
    #return一個從現在到以前的日期列表，用來作為csv文件的head的一部分，之後每一個日期的數據只佔一格
#     head = ["name","institution","city","pgp","email","twitter","website","github"]
    head = ["name","institution","city","github"]
    today = time.strftime("%Y-%m-%d")
    head.extend(dateRange(beginDate, today))
    return head





with open('journalists-data.csv','r') as f:  #讀取上一部分輸出的記者基本信息表
    with open('journalists-data-including-github-contributions.csv','a') as d:
        csv.writer(d).writerow(write_head('2007-12-30')) #github是08年發布的，但顯示最早到20171230，所以從這個日期開始爬
        for row in csv.reader(f):
            try:
                row.extend(scrape_github_contributions(row[3])) #row[3]是每位記者的github地址
                csv.writer(d).writerow(row)
            except:
                row.append('N/A') 
                #疑問：對於沒有github的記者，應該所有日期的contribution全部標註為N/A還是0？這個可以之後再定
                csv.writer(d).writerow(row)

CPU times: user 56.2 s, sys: 1.34 s, total: 57.5 s
Wall time: 25min 55s


In [59]:
import pandas as pd
import numpy as np
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
df=pd.read_csv('journalists-data-including-github-contributions.csv')
df=df[np.isnan(df['2018-11-04']) == False].T #2018-11-04如果為N/A，則說明此人沒有github
df.to_csv('Result.csv') #寫出的文件中contributions是float，還不知道怎麼變成整數
df

,1,4,7,8,11,14,15,16,17,18,19,20,21,22,23,24,25,28,29,32,33,34,36,37,39,41,42,43,47,48,50,55,56,58,62,75,83,84,89,91,99,100,101,102,103,104,106,109,110,118,119,122,127,128,132,144,152,156,162,163,164,165,166,167,172,175,177,178,186,187,189,190,192,196,202,205,206,207,208,209,213
name,Justin Myers,Annabel Church,Mina Demian,Martín González,Adolfo Antón Bravo,Michael Mott,Troy Thibodeaux,John Heasly,Tom Cardoso,Jeremia Kimelman,Vanessa Wormer,Leonard Wallentin,Julien Gathelier,Stanislaw M. Stanuch,Tom Meagher,Edouard Richard,Maureen Linke,Thomas Preusse,Alexandre Haederli,Peter Aldhous,Jeremy Rue,Cory Kendrick,David Barreda,Moiz Syed,Llewellyn Hinkes-Jones,Erin Richey,Tony Papousek,Hunter Owens,Konrad Weber,Ricardo Lafuente,Yuichi Yazaki,Danielle Alberti,Ben Casselman,Eduard Martín-Borregón,Gary Dagorn,Bernd Oswald,Andrea Angeli,Giuseppe Futia,Alessio Cimarelli,Masayuki Shimizu,Chad Skelton,Concha Catalan,Harkanwal Singh,Moritz Klack,Sylke Gruhnwald,Michael Hörz,David Bauer,Jonathon Morgan,Claus Hesseling,Friedrich Lindenberg,Natalia Karbasova,Duc-Quang Nguyen,Bruno De Bondt,Jonathan Stray,Markus Hametner,John Swain,Marcus Schwarze,Jesús Escudero,Juan Elosua,Antonio Delgado,Federico Comesaña,Matthew Caruana Galizia,Mar Cabra,Steven Rich,Eva Constantaras,Steven Melendez,Justin Myers,Ana Isabel Carvalho,Youyou Zhou,Pinar Dag,Julian Schmidli,Mona Chalabi,Jean Abbiateci,Raphaël da Silva,Mehdi Guiraud,Alexandre Léchenet,Paul Joannon,Sebastian Kraus,Pierre Bellon,Pierre Romera,Cosmin Cabulea
institution,The Associated Press,NaN,Africa Check,Freelance,Infotics,The Willits News,The Associated Press,The Register-Guard,The Globe and Mail,Code for America,Heilbronner Stimme,Journalism++ Stockholm,Freelance,małopolskie,The Marshall Project,Journalism++,The Associated Press,Neue Zürcher Zeitung,Le Matin Dimanche,Freelance,UC Berkeley,NaN,ChinaFile,NaN,NaN,Freelance,NaN,NaN,SRF,Journalism++ Porto,NaN,NaN,FiveThirtyEight,Data'n'Press,Ouest-France,Freelance,NaN,Nexa Center for Internet & Society,Dataninja.it,NaN,The Vancouver Sun,Opengov.cat,New Zealand Herald,webkid,Neue Zürcher Zeitung,Freelance,NaN,CrisisNET,Freelance,ICFJ,Hubert Burda Media,SWISSINFO,Freelance,Columbia University,DER STANDARD,Kandula Ltd,Rhein-Zeitung,El Confidencial,Freelance,Freelance,El Observador,ICIJ,ICIJ,The Washington Post,Internews,Freelance,The Chronicle of Higher Education,Journalism++ Porto,Graphicacy,Dag Medya,Tages-Anzeiger,FiveThirtyEight,NaN,Rue89Strasbourg,NaN,Le Monde,Journalism++,Journalism++,Journalism++,Journalism++,Freelance
city,"United States, Brooklyn (US-VA)","N/A, Germany","Johannesburg, South Africa","Barcelona, Spain","Madrid, Spain","Willits, United States","New Orleans, United States","Eugene, United States","Toronto, Canada","San Francisco, United States","Heilbronn, Germany","Stockholm, Sweden","New York, United States","Kraków, Poland","New York, United States","Paris, France","Arlington (US-VA), United States","Zurich, Switzerland","Berne, Switzerland","San Francisco, United States","Berkeley, United States","San Francisco, United States","New York, United States","N/A, United States","Washington, D.C., United States","St. Louis, United States","Lincoln (US-NE), United States","Chicago, United States","Zurich, Switzerland","Porto, Portugal","Tokyo, Japan","Washington, D.C., United States","New York, United States","Barcelona, Spain","Caen, France","Munich, Germany","Siena, Italy","N/A, Italy","Squillace, Italy","Gunma, Japan","Vancouver, Canada","Barcelona, Spain","Auckland, New Zealand","Berlin, Germany","Zurich, Switzerland","Berlin, Germany","Basel, Switzerland","Austin, United States","Hamburg, Germany","Cape Town, South Africa","Munich, Germany","Lausanne, Switzerland","Vancouver, Canada","New York, United States","Vienna, Austria","Leeds, United Kingdom","Koblenz, Germany","Madrid, Spain","Vigo, Spain","Madrid, Spain","Montevideo, Uruguay","San José, Costa Rica","Madrid, Spain","Washington, D.C., United States","Nairobi, Kenya","New Y